In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
set_mean = 10
indata = pd.read_csv('../data/hallem2006_TableS1_source.csv', index_col=0, header=0, encoding='utf-8')
normed = indata.T - indata.T.min()
normed = set_mean * normed/normed.mean()
normed.drop(['spontaneous firing rate'], axis=1, inplace=True)
normed

# Distribution of elements
Looks exponential, but there are correlations between odors, I'm sure, because dot products below do not, at all, follow the distribution of dot products between vectors made of iid exponential elements. 

In [ ]:
fig, ax = plt.subplots()
ax.hist(normed.values.flatten(), bins="fd")
ax.set(xlabel="ORN activation", ylabel="Count")
plt.show()
plt.close()

# Study dot products

In [ ]:
unitnormed = normed / np.sqrt((normed**2).sum(axis=0))
dot_products = unitnormed.T.dot(unitnormed)

In [ ]:
dot_products_stats = dot_products.values[np.triu_indices(n=dot_products.shape[0], k=1)]

In [ ]:
fig, ax = plt.subplots()
ax.hist(dot_products_stats, bins="fd")
ax.set(xlabel="Dot product", ylabel="Count")
plt.show()
plt.close()

# Conclusion: odors are not typically orthogonal! 
This is a big problem for the inhibition problem I have been considering, because the more orthogonal odors are, the better it works. 

The reason is the non-negativity of those vectors. Random non-negative vectors won't be orthogonal on average. And real odors tend to have similarity due to chemical similarities, so it's even worse here. 

Maybe by combining odors with Gautam's model, we can circumvent that problem, but for now, it's a serious issue. 

# Check random non-negative vectors
Uniformly random, then exponential random. 

In [ ]:
rgen = np.random.default_rng(seed=0x5296381dd940500416b73afc46718ddf)

In [ ]:
n_dims, n_odors = normed.shape
random_uniform_nn_vectors = rgen.random(size=(n_dims, n_odors))
random_uniform_nn_vectors /= np.sqrt(np.sum(random_uniform_nn_vectors**2, axis=0, keepdims=True))
dot_products_uniform = random_uniform_nn_vectors.T.dot(random_uniform_nn_vectors)
dot_products_uniform_stats = dot_products_uniform[np.triu_indices(n=n_odors, k=1)]

In [ ]:
random_expo_nn_vectors = rgen.exponential(size=(n_dims, n_odors))
random_expo_nn_vectors /= np.sqrt(np.sum(random_expo_nn_vectors**2, axis=0, keepdims=True))
dot_products_expo = random_expo_nn_vectors.T.dot(random_expo_nn_vectors)
dot_products_expo_stats = dot_products_expo[np.triu_indices(n=n_odors, k=1)]

In [ ]:
# Log-normal: depends a lot on the sigma parameter. 
# Small sigma: all vectors look alike, dot products are close to 1
# Large sigma: each vector has a few large components from the long tail, dot products are close to 0
#random_lognorm_nn_vectors = rgen.lognormal(sigma=1.0, size=(n_dims, n_odors))
#random_lognorm_nn_vectors /= np.sqrt(np.sum(random_lognorm_nn_vectors**2, axis=0, keepdims=True))
#dot_products_lognorm = random_lognorm_nn_vectors.T.dot(random_lognorm_nn_vectors)
#dot_products_lognorm_stats = dot_products_lognorm[np.triu_indices(n=n_odors, k=1)]

In [ ]:
fig, ax = plt.subplots()
ax.hist(dot_products_stats, bins="fd", label="Data", alpha=0.7)
ax.hist(dot_products_uniform_stats, bins="fd", label="Uniform", alpha=0.7)
ax.hist(dot_products_expo_stats, bins="fd", label="Exponential", alpha=0.7)
#ax.hist(dot_products_lognorm_stats, bins="fd", label="Log-normal", alpha=0.7)
ax.set(xlabel="Dot product", ylabel="Count")
ax.legend()
plt.show()
plt.close()